In [1]:
#data processing
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time

#data visualization
%matplotlib inline
import matplotlib.pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
##### import local functions
# for data processing
from utilities_NSF_EC2022 import create_url, get_data_from_url, get_info_from_df,get_data_for_timeRange, \
                                    polygon_lon_lat, mask_QC, simple_plot, qc, interpolate
# for data visualization
from utilities_NSF_EC2022 import set_ax_label, \
                                 set_map_and_plot_locations_withColor

In [6]:
# prefix to use with all API queries
URL_PREFIX = 'https://argovis-api.colorado.edu'
API_KEY   = ''

In [ ]:
startDate = '2021-04-20T00:00:00Z'
endDate   = '2021-05-02T00:00:00Z'
# To see what this polygon looks like, visit https://argovis.colorado.edu/ng/home?mapProj=WM&presRange=%5B0,2000%5D&selectionStartDate=2022-03-31T22:30:45Z&selectionEndDate=2022-04-14T22:30:45Z&threeDayEndDate=2022-04-12T22:30:45&shapes=%5B%5B%5B22.105999,-76.289063%5D,%5B26.902477,-72.597656%5D,%5B26.084682,-66.395555%5D,%5B25.005973,-60.292969%5D,%5B10.833306,-65.566406%5D,%5B11.942098,-72.133142%5D,%5B22.105999,-76.289063%5D%5D%5D&includeRealtime=true&onlyBGC=false&onlyDeep=false&threeDayToggle=false
polygon = '[[-76.289063,22.105999],[-72.597656,26.902477],[-66.395555,26.084682],[-60.292969,25.005973],[-65.566406,10.833306],[-72.133142,11.942098],[-76.289063,22.105999]]'

# helper function parameters
## list of parameters to extract from dataframes to dictionaries for plotting, see get_info_from_df
info_to_store = ['lon','lat','date','cols_bySource','ids','woce_line']

## repackaging of polygon coordinates for plotting, see set_map_and_plot_locations_withColor
polygon_lon_lat_dict = polygon_lon_lat(polygon_str=polygon)

In [ ]:
source_all = ['argo_core','argo_bgc','cchdo_go-ship']
for isource in source_all:
    bfr_df = get_data_for_timeRange(startDate=startDate,endDate=endDate, \
                                url_prefix=URL_PREFIX+'/profiles?', \
                                myAPIkey=API_KEY, \
                                source=isource, \
                                polygon=polygon, \
                                dt_tag='365d',writeFlag=True)
    if not bfr_df.empty:
        bfr_info = get_info_from_df(df=bfr_df,info_to_store=info_to_store)
        ###### let's plot the polygon and the profile locations color coded by source
        set_map_and_plot_locations_withColor(lon=bfr_info['lon'],lat=bfr_info['lat'], \
                                     cols=bfr_info['cols_bySource'], \
                                     polygon_lon_lat_dict=polygon_lon_lat_dict)
        plt.title('source: ' + isource,fontsize=28)

In [ ]:
bfr_df = get_data_for_timeRange(startDate=startDate,endDate=endDate, \
                            url_prefix=URL_PREFIX+'/profiles?', \
                            myAPIkey=API_KEY, \
                            source='argo_core,~argo_bgc', \
                            polygon=polygon, \
                            dt_tag='365d',writeFlag=True)
if not bfr_df.empty:
    bfr_info = get_info_from_df(df=bfr_df,info_to_store=info_to_store)
    ###### let's plot the polygon and the profile locations color coded by source
    set_map_and_plot_locations_withColor(lon=bfr_info['lon'],lat=bfr_info['lat'], \
                                 cols=bfr_info['cols_bySource'], \
                                 polygon_lon_lat_dict=polygon_lon_lat_dict)
    plt.title('source: ' + isource,fontsize=28)

In [ ]:
df = get_data_for_timeRange(startDate=startDate,endDate=endDate, \
                                url_prefix=URL_PREFIX+'/profiles?', \
                                myAPIkey=API_KEY, \
                                data='doxy,pres,metadata-only', \
                                polygon=polygon, \
                                dt_tag='100d',writeFlag=True)
if not df.empty:
    if 'df_all' in locals():
        df_all = pd.concat([df_all,df],ignore_index=True)
    else:
        df_all = df
            
# store some info
info_d_reg = get_info_from_df(df=df_all,info_to_store=info_to_store)

# delete df_all
del df_all

###### let's plot the polygon and profile locations color coded by source
set_map_and_plot_locations_withColor(lon=info_d_reg['lon'],lat=info_d_reg['lat'], \
                                     cols=info_d_reg['cols_bySource'], \
                                     polygon_lon_lat_dict=polygon_lon_lat_dict)

In [ ]:
if 'df_d1' in locals():
    del df_d1

df = get_data_for_timeRange(startDate=startDate,endDate=endDate, \
                                url_prefix=URL_PREFIX+'/profiles?', \
                                myAPIkey=API_KEY, \
                                data='pres,doxy_ctd,metadata-only', \
                                woceline='A22', \
                                dt_tag='365d',writeFlag=True)
if not df.empty:
    if 'df_d1' in locals():
        df_d1 = pd.concat([df_d1,df],ignore_index=True)
    else:
        df_d1 = df
# store some info
info_d1 = get_info_from_df(df=df_d1,info_to_store=info_to_store)

font = {'weight' : 'normal',
        'size'   : 24}

plt.rc('font', **font)
set_map_and_plot_locations_withColor(lon=info_d1['lon'],lat=info_d1['lat'], \
                                     cols=info_d1['cols_bySource'], \
                                     polygon_lon_lat_dict=[])
plt.title('A22',fontsize=28)

In [ ]:
delta_days = 3.5

if 'df_d2' in locals():
    del df_d2

lst_id_pairs_d1 = []
lst_id_pairs_d2 = []

for i in np.arange(0,len(info_d1['lon'])):
    time.sleep(.2)
    d = get_data_for_timeRange(startDate=(datetime.strptime(info_d1['date'][i],'%Y-%m-%dT%H:%M:%SZ')- \
                                          timedelta(days=delta_days)).strftime('%Y-%m-%dT%H:%M:%SZ'),
                               endDate=(datetime.strptime(info_d1['date'][i],'%Y-%m-%dT%H:%M:%SZ')+ \
                                          timedelta(days=delta_days)).strftime('%Y-%m-%dT%H:%M:%SZ'), \
                               myAPIkey=API_KEY, \
                               center=str(info_d1['lon'][i])+','+str(info_d1['lat'][i]), \
                               radius_km='50', \
                               url_prefix=URL_PREFIX+'/profiles?', \
                               data='pres,doxy,metadata-only', \
                               source='argo_core', \
                               dt_tag='365d',writeFlag=False)
    if not d.empty:
        #info_d2 = {**info_d2, **get_info_from_df(df=d,info_to_store=info_to_store)}
        #list_d2 = list_d2 + [d]
        if 'df_d2' in locals():
            df_d2 = pd.concat([df_d2,d],ignore_index=True)
        else:
            df_d2 = d
        lst_id_pairs_d2.append(d['_id'].tolist())
        lst_id_pairs_d1.append(info_d1['ids'][i])

info_d2=get_info_from_df(df=df_d2,info_to_store=info_to_store)

###### let's plot the profile locations color coded by source
set_map_and_plot_locations_withColor(lon=info_d1['lon']+info_d2['lon'],lat=info_d1['lat']+info_d2['lat'], \
                                     cols=info_d1['cols_bySource']+info_d2['cols_bySource'])

In [ ]:
font = {'weight' : 'normal',
        'size'   : 24}

plt.rc('font', **font)

col_d1 = 'k'
col_d2 = 'r'

var_d = ['doxy_ctd', 'doxy']

mrk_sz1 = 7
mrk_sz2 = 5

mrk_type1 = '*'
mrk_type2 = 's'

presRange ='0,100'

for i in np.arange(0,len(lst_id_pairs_d1),1):
    ids_bfr1 = lst_id_pairs_d1[i]
    ids_bfr2 = lst_id_pairs_d2[i]
    if isinstance(ids_bfr1,str):
        ids_bfr1 = [ids_bfr1]
    if isinstance(ids_bfr2,str):
        ids_bfr2 = [ids_bfr2]
        
    fig = plt.figure(figsize=(20,10))
    fig_close = True
    
    if 'bfr_title' in locals():
        del bfr_title
        
    # plot 
    for isource in [1,2]:
        if 'ids_bfr' in locals():
            del ids_bfr
        ids_bfr = eval('ids_bfr'+str(isource))
        for j in ids_bfr:
            bfr = pd.DataFrame(get_data_from_url(url=URL_PREFIX+'/profiles?&data='+var_d[isource-1]+',pres&id='+j+'&presRange='+presRange,\
                                                 myAPIkey=API_KEY,writeFlag=False))
            if 'bfr_df' in locals():
                del bfr_df

            for j_df in bfr['data']:
                if not 'bfr_df' in locals():
                    bfr_df = pd.DataFrame(j_df)
                else:
                    bfr_df = pd.concat([bfr_df,j_df],ignore_index=True)
            if var_d[isource-1] in bfr_df.keys():
                bfr_df2plt = bfr_df[bfr_df.eval(var_d[isource-1]).notnull()]
                plt.plot(bfr_df2plt[var_d[isource-1]],bfr_df2plt['pres'], \
                         marker=eval('mrk_type'+str(isource)),markersize=eval('mrk_sz'+str(isource)),color=eval('col_d'+str(isource)),linewidth=2)
                if isource ==2:
                    fig_close = False
                if not 'bfr_title' in locals():
                    bfr_title = j 
                else:
                    bfr_title = bfr_title + ', ' + j 
    plt.gca().invert_yaxis()
    plt.title(bfr_title)
    plt.xlabel(set_ax_label(var_d[0]))
    plt.ylabel(set_ax_label('pres'))
    if fig_close:
        plt.close(fig)